In [40]:
# Importing Dependencies
import numpy as np
import gym
from gym.envs.box2d.car_racing import *
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization
from keras.optimizers import Adam
from statistics import mean
from pyglet.window import key
import time
from tqdm import tqdm 
import os
from keras.utils import normalize

In [2]:
# Function for generating data by playing
def start_playing(n_episodes):
    a = np.array( [0.0, 0.0, 0.0] )
    def key_press(k, mod):
        global restart
        if k==0xff0d: restart = True
        if k==key.LEFT:  a[0] = -1.0
        if k==key.RIGHT: a[0] = +1.0
        if k==key.UP:    a[1] = +1.0
        if k==key.DOWN:  a[2] = +0.8   
    def key_release(k, mod):
        if k==key.LEFT  and a[0]==-1.0: a[0] = 0
        if k==key.RIGHT and a[0]==+1.0: a[0] = 0
        if k==key.UP:    a[1] = 0
        if k==key.DOWN:  a[2] = 0
    
    env = CarRacing()
    env.render()
    env.viewer.window.on_key_press = key_press
    env.viewer.window.on_key_release = key_release
    
    for episode in range(n_episodes):
        prev_observation = env.reset()
        if episode==0:
            X = np.array(prev_observation.tolist())
            y = np.array([0.,0.,0.])
        restart = False
        while True:
            action = a
            observation, reward, done, info = env.step(action)
            X = np.vstack((X, prev_observation))
            y = np.vstack((y, action))
            prev_observation=observation
            env.render()
            if done or restart: break
        time.sleep(2)    
    np.save('train_X.npy', X)
    np.save('train_y.npy', y)    
    env.close()
    return X, y

In [3]:
# Playing the to create data
X, y=start_playing(20)

Track generation: 1121..1405 -> 284-tiles track
Track generation: 925..1162 -> 237-tiles track
retry to generate track (normal if there are not many of this messages)
Track generation: 1228..1538 -> 310-tiles track
Track generation: 1132..1419 -> 287-tiles track
Track generation: 1282..1607 -> 325-tiles track
Track generation: 1171..1472 -> 301-tiles track
retry to generate track (normal if there are not many of this messages)
Track generation: 1120..1404 -> 284-tiles track
Track generation: 1030..1291 -> 261-tiles track
Track generation: 1119..1403 -> 284-tiles track
Track generation: 1126..1412 -> 286-tiles track
Track generation: 1413..1770 -> 357-tiles track
Track generation: 1230..1541 -> 311-tiles track
Track generation: 1246..1562 -> 316-tiles track
Track generation: 1128..1414 -> 286-tiles track
Track generation: 1085..1365 -> 280-tiles track
Track generation: 1387..1738 -> 351-tiles track
Track generation: 992..1251 -> 259-tiles track
Track generation: 1029..1296 -> 267-tiles 

In [8]:
# Combining gameplay files
temp = np.load('train_data-549.4999999999591.npy')
tempX = [i[0] for i in temp]
tempy = [i[1] for i in temp]
X = np.array(tempX)
y = np.array(tempy)
for file in tqdm(os.listdir('Data/')):
    temp = np.load('Data/' + file)
    tempX = [i[0] for i in temp]
    tempy = [i[1] for i in temp]
    tempX = np.array(tempX)
    tempy = np.array(tempy)
    X  = np.concatenate([X, tempX], axis=0)
    y  = np.concatenate([y, tempy], axis=0)

np.save('X.npy', X)    

100%|████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:05<00:00,  1.60it/s]


In [5]:
# Converting output to proper OneHot array
Y=[]
for i in range(len(y)):  
    if np.array_equal(y[i], np.array([0,1,0])):
        Y.append(np.array([0,0,1,0,0]))
    elif np.array_equal(y[i], np.array([0,0,0.8])):
        Y.append(np.array([0,0,0,1,0]))
    elif np.array_equal(y[i], np.array([1,0,0])):
        Y.append(np.array([0,1,0,0,0]))
    elif np.array_equal(y[i], np.array([-1,0,0])):
        Y.append(np.array([1,0,0,0,0]))
    else:
        Y.append(np.array([0,0,0,0,1]))      
      
np.save('y_modified.npy', Y)    

In [41]:
# Loading the data
X = np.load('train_X.npy')
y = np.load('y_modified.npy')
X = X.reshape(-1,96,96,3)
print(X.shape)
print(y.shape)

(20450, 96, 96, 3)
(20450, 5)


In [23]:
# Variation in generated data
left=np.array([1,0,0,0,0])
right=np.array([0,1,0,0,0])
straight=np.array([0,0,1,0,0])
stop=np.array([0,0,0,1,0])
no_key=np.array([0,0,0,0,1])
l=0
r=0
s=0
b=0
nk=0

for key in y:
    if np.array_equal(key, left):
        l+=1
    elif np.array_equal(key, right):
        r+=1
    elif np.array_equal(key, straight):
        s+=1
    elif np.array_equal(key, stop):
        b+=1
    elif np.array_equal(key, no_key):
        nk+=1    
      
print(l,r,s,b,nk)    

2694 1318 4443 179 11816


In [56]:
model = Sequential()

model.add(Conv2D(filters = 16, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu', input_shape = (96,96,3)))
model.add(Conv2D(filters = 16, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.5))

model.add(Conv2D(filters = 32, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(Conv2D(filters = 32, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.5))


model.add(Flatten())
model.add(Dense(256, activation = "relu"))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(5, activation = "softmax"))

In [57]:
optimizer = Adam(lr=0.0001)
model.compile(optimizer = optimizer, loss = "categorical_crossentropy")

In [58]:
model.fit(X, y, epochs=10, validation_split=0.2, batch_size=600, verbose=1, shuffle=True)

Train on 16360 samples, validate on 4090 samples
Epoch 1/10
16360/16360 [==============================] - ETA: 53s - loss: 2.71 - ETA: 29s - loss: 2.62 - ETA: 21s - loss: 2.62 - ETA: 17s - loss: 2.57 - ETA: 14s - loss: 2.52 - ETA: 13s - loss: 2.52 - ETA: 11s - loss: 2.51 - ETA: 10s - loss: 2.49 - ETA: 9s - loss: 2.4639 - ETA: 8s - loss: 2.445 - ETA: 7s - loss: 2.415 - ETA: 7s - loss: 2.397 - ETA: 6s - loss: 2.383 - ETA: 6s - loss: 2.367 - ETA: 5s - loss: 2.350 - ETA: 4s - loss: 2.344 - ETA: 4s - loss: 2.343 - ETA: 3s - loss: 2.335 - ETA: 3s - loss: 2.331 - ETA: 3s - loss: 2.322 - ETA: 2s - loss: 2.312 - ETA: 2s - loss: 2.301 - ETA: 1s - loss: 2.286 - ETA: 1s - loss: 2.279 - ETA: 0s - loss: 2.267 - ETA: 0s - loss: 2.254 - ETA: 0s - loss: 2.248 - 12s 736us/step - loss: 2.2458 - val_loss: 1.8947
Epoch 2/10
16360/16360 [==============================] - ETA: 8s - loss: 1.943 - ETA: 8s - loss: 1.937 - ETA: 7s - loss: 1.924 - ETA: 7s - loss: 1.913 - ETA: 7s - loss: 1.931 - ETA: 7s - loss: 1

In [ ]:
# Saving the Model
model.save('Models/first_model')

In [ ]:
# Loading the Model
model.load('Models/first-model')

In [73]:
#scores = []
#choices = []
env = CarRacing()
for each_game in range(3):
    #score = 0
    #game_memory = []
    prev_obs = env.reset()
    for i in range(1000):
        env.render()
        out = model.predict(prev_obs.reshape(-1,96,96,3))[0]
        print(out)
        
        if out[0]<=0.35:
            out[0]=0  
        if out[1]<=0.2:
            out[1]=0
        if out[2]<=0.01 :
            out[2]=0   
        if out[3]<=0.47:
            out[3]=0
        if out[4]<=9:
            out[4]=0    
        
        output = np.argmax(out)
        action = np.zeros(5)
        np.put(action, output, 1)
        #choices.append(action)
        if np.array_equal(action, np.array([0,0,1,0,0])):
            action=np.array([0,1,0])
        elif np.array_equal(action, np.array([0,0,0,1,0])):
            action=np.array([0,0,0.8])
        elif np.array_equal(action, np.array([0,1,0,0,0])):
            action=np.array([1,0,0])
        elif np.array_equal(action, np.array([1,0,0,0,0])):
            action=np.array([-1,0,0])
        else:
            action=np.array([0,0,0,0,0])
        
        new_observation, reward, done, info = env.step(action)
        print(action,"by nn")
        prev_obs = new_observation
        #game_memory.append([new_observation, action])
        #score+=reward
        if done: 
            break
    #scores.append(score)
    #print('Episode {} Score: {}'.format(each_game+1,score))
    env.close()
#print('Average Score:',sum(scores)/len(scores))

Track generation: 1380..1729 -> 349-tiles track
[1.6442436e-04 2.2818191e-05 2.5397397e-03 1.0105422e-06 9.9727196e-01]
[-1  0  0] by nn
[3.52858931e-01 1.72678918e-01 3.48536996e-05 1.06162526e-01
 3.68264735e-01]
[-1  0  0] by nn
[8.3430775e-02 6.4376348e-01 3.6909637e-07 1.4804999e-02 2.5800034e-01]
[1 0 0] by nn
[2.7686701e-04 2.5917229e-01 2.8544762e-09 2.9073379e-03 7.3764348e-01]
[1 0 0] by nn
[6.1393544e-06 7.4700988e-03 6.8491657e-10 5.9846789e-04 9.9192530e-01]
[-1  0  0] by nn
[4.2761651e-05 1.8267358e-02 3.2690500e-10 3.4909819e-03 9.7819889e-01]
[-1  0  0] by nn
[3.8040183e-05 1.6056962e-02 4.3465129e-09 1.1993162e-03 9.8270571e-01]
[-1  0  0] by nn
[1.71740372e-02 1.10818565e-01 1.59154888e-06 3.72896120e-02
 8.34716201e-01]
[-1  0  0] by nn
[4.31500934e-03 1.04166046e-01 7.73821739e-06 3.16188261e-02
 8.59892368e-01]
[-1  0  0] by nn
[1.5594886e-02 2.2372605e-01 1.9868572e-05 1.6669667e-01 5.9396255e-01]
[1 0 0] by nn
[2.8853422e-02 6.7119919e-02 1.2578010e-05 9.6369669e

[0.86147386 0.01728413 0.00438677 0.09169513 0.02516014]
[-1  0  0] by nn
[0.902813   0.01143992 0.00405471 0.06596035 0.01573203]
[-1  0  0] by nn
[0.92813116 0.00824606 0.00340045 0.05010948 0.01011284]
[-1  0  0] by nn
[0.9422257  0.00767106 0.00283498 0.03997673 0.00729163]
[-1  0  0] by nn
[0.9498184  0.00583527 0.00256536 0.03526185 0.00651911]
[-1  0  0] by nn
[0.95736915 0.00372999 0.00210983 0.02935561 0.00743539]
[-1  0  0] by nn
[0.96268547 0.00274461 0.00207728 0.02263113 0.00986157]
[-1  0  0] by nn
[0.9584194  0.00259058 0.00252692 0.02393763 0.01252548]
[-1  0  0] by nn
[0.9579347  0.00222674 0.00260307 0.02122363 0.01601186]
[-1  0  0] by nn
[0.9480753  0.00248649 0.00314093 0.02110278 0.02519447]
[-1  0  0] by nn
[0.9394969  0.00268584 0.00316858 0.02203158 0.03261719]
[-1  0  0] by nn
[0.9251144  0.00279104 0.0033751  0.0255555  0.04316406]
[-1  0  0] by nn
[0.91627455 0.00366425 0.00415148 0.02855566 0.04735403]
[-1  0  0] by nn
[0.8902862  0.00670633 0.00730935 0.03

[0.83124197 0.00730321 0.00857822 0.03821076 0.11466583]
[-1  0  0] by nn
[0.83124524 0.00566796 0.00837427 0.04913115 0.10558127]
[-1  0  0] by nn
[0.89404225 0.00325165 0.00258542 0.03387998 0.06624063]
[-1  0  0] by nn
[0.89009273 0.00303483 0.00295934 0.03432702 0.06958614]
[-1  0  0] by nn
[0.91326356 0.0038512  0.00193643 0.02351263 0.05743631]
[-1  0  0] by nn
[0.9121719  0.00410465 0.00229975 0.02316679 0.05825686]
[-1  0  0] by nn
[0.8942213  0.00515931 0.00443735 0.02866296 0.06751908]
[-1  0  0] by nn
[0.8910106  0.00793569 0.00679902 0.03267138 0.06158344]
[-1  0  0] by nn
[0.87923527 0.01085331 0.01192016 0.03285691 0.06513433]
[-1  0  0] by nn
[0.7563993  0.0240909  0.04033165 0.05329049 0.12588769]
[-1  0  0] by nn
[0.5720524  0.05630963 0.08800645 0.08905669 0.19457485]
[-1  0  0] by nn
[0.3022453  0.11449333 0.1263833  0.14959028 0.3072878 ]
[0 1 0] by nn
[0.10534015 0.21078189 0.11400131 0.1389725  0.4309041 ]
[1 0 0] by nn
[0.03587022 0.38884246 0.0816846  0.08238476

 3.92711739e-31]
[-1  0  0] by nn
[9.9999845e-01 2.9749654e-09 3.2524232e-14 1.5309365e-06 1.6279681e-32]
[-1  0  0] by nn
[9.9999845e-01 2.4792843e-09 1.2707287e-14 1.4927308e-06 1.0115612e-32]
[-1  0  0] by nn
[9.9999893e-01 1.2103925e-09 1.4732427e-14 1.1044377e-06 2.9165391e-33]
[-1  0  0] by nn
[9.9999917e-01 1.8732098e-09 3.3013072e-14 8.8980437e-07 1.0348949e-32]
[-1  0  0] by nn
[9.9999821e-01 2.7321896e-09 1.4763562e-13 1.7377469e-06 2.8383910e-31]
[-1  0  0] by nn
[9.9999535e-01 6.8996795e-09 1.0309258e-12 4.6265022e-06 1.1189643e-29]
[-1  0  0] by nn
[9.9999452e-01 1.1679773e-08 3.9717830e-12 5.4257675e-06 1.2832558e-28]
[-1  0  0] by nn
[9.9999261e-01 2.6656087e-08 1.3934487e-11 7.3758274e-06 3.0711293e-27]
[-1  0  0] by nn
[9.9999070e-01 4.2607031e-08 5.6755219e-11 9.3323179e-06 1.0821871e-25]
[-1  0  0] by nn
[9.9998808e-01 6.1274555e-08 1.3837521e-10 1.1773072e-05 8.9428230e-25]
[-1  0  0] by nn
[9.9998617e-01 1.6045463e-07 8.5953583e-10 1.3747428e-05 3.1877102e-23]
[-1 

[9.9999833e-01 9.0186703e-08 6.0750370e-11 1.5012151e-06 1.2035242e-25]
[-1  0  0] by nn
[9.9999928e-01 3.0211094e-08 4.0321791e-12 7.6033990e-07 2.7765009e-27]
[-1  0  0] by nn
[9.9999905e-01 6.2782675e-08 2.1098793e-11 8.5828600e-07 1.4458210e-24]
[-1  0  0] by nn
[9.9999225e-01 1.2185528e-06 1.5696077e-09 6.5610575e-06 1.4697496e-18]
[-1  0  0] by nn
[9.9985552e-01 1.5568194e-05 2.8656951e-08 1.2877563e-04 4.5150572e-12]
[-1  0  0] by nn
[9.9821424e-01 1.5053333e-04 8.6348237e-08 1.6348275e-03 3.5732370e-07]
[-1  0  0] by nn
[9.9555004e-01 2.7297233e-04 4.2536379e-07 4.1024419e-03 7.4116673e-05]
[-1  0  0] by nn
[9.8834538e-01 7.1322458e-04 3.4134268e-06 8.5153664e-03 2.4225698e-03]
[-1  0  0] by nn
[9.8004454e-01 1.5632446e-03 9.6556454e-05 1.6157333e-02 2.1383113e-03]
[-1  0  0] by nn
[9.7004557e-01 2.2103626e-03 5.5411621e-04 2.6378129e-02 8.1186916e-04]
[-1  0  0] by nn
[9.2795926e-01 5.2870242e-03 7.1855477e-04 6.5046310e-02 9.8881265e-04]
[-1  0  0] by nn
[9.2262620e-01 1.0105

...

[0.08919033 0.31526783 0.02715745 0.5404071  0.02797727]
[0.  0.  0.8] by nn
[0.08919033 0.31526783 0.02715745 0.5404071  0.02797727]
[0.  0.  0.8] by nn
[0.08919033 0.31526783 0.02715745 0.5404071  0.02797727]
[0.  0.  0.8] by nn
[0.08919033 0.31526783 0.02715745 0.5404071  0.02797727]
[0.  0.  0.8] by nn
[0.08919033 0.31526783 0.02715745 0.5404071  0.02797727]
[0.  0.  0.8] by nn
[0.07871284 0.27849683 0.02830895 0.58581    0.02867141]
[0.  0.  0.8] by nn
[0.07871284 0.27849683 0.02830895 0.58581    0.02867141]
[0.  0.  0.8] by nn
[0.07871284 0.27849683 0.02830895 0.58581    0.02867141]
[0.  0.  0.8] by nn
[0.07871284 0.27849683 0.02830895 0.58581    0.02867141]
[0.  0.  0.8] by nn
[0.07871284 0.27849683 0.02830895 0.58581    0.02867141]
[0.  0.  0.8] by nn
[0.07871284 0.27849683 0.02830895 0.58581    0.02867141]
[0.  0.  0.8] by nn
[0.07871284 0.27849683 0.02830895 0.58581    0.02867141]
[0.  0.  0.8] by nn
[0.07871284 0.27849683 0.02830895 0.58581    0.02867141]
[0.  0.  0.8] by nn

[0.07691409 0.28915945 0.02857344 0.576145   0.02920802]
[0.  0.  0.8] by nn
[0.07691409 0.28915945 0.02857344 0.576145   0.02920802]
[0.  0.  0.8] by nn
[0.07691409 0.28915945 0.02857344 0.576145   0.02920802]
[0.  0.  0.8] by nn
[0.07691409 0.28915945 0.02857344 0.576145   0.02920802]
[0.  0.  0.8] by nn
[0.07691409 0.28915945 0.02857344 0.576145   0.02920802]
[0.  0.  0.8] by nn
[0.07691409 0.28915945 0.02857344 0.576145   0.02920802]
[0.  0.  0.8] by nn
[0.07691409 0.28915945 0.02857344 0.576145   0.02920802]
[0.  0.  0.8] by nn
[0.07691409 0.28915945 0.02857344 0.576145   0.02920802]
[0.  0.  0.8] by nn
[0.07691409 0.28915945 0.02857344 0.576145   0.02920802]
[0.  0.  0.8] by nn
[0.08577374 0.260503   0.03187441 0.5957328  0.02611604]
[0.  0.  0.8] by nn
[0.08577374 0.260503   0.03187441 0.5957328  0.02611604]
[0.  0.  0.8] by nn
[0.08577374 0.260503   0.03187441 0.5957328  0.02611604]
[0.  0.  0.8] by nn
[0.08577374 0.260503   0.03187441 0.5957328  0.02611604]
[0.  0.  0.8] by nn

[0.09301588 0.3041357  0.02999867 0.5479954  0.02485436]
[0.  0.  0.8] by nn
[0.09301588 0.3041357  0.02999867 0.5479954  0.02485436]
[0.  0.  0.8] by nn
[0.09301588 0.3041357  0.02999867 0.5479954  0.02485436]
[0.  0.  0.8] by nn
[0.08561901 0.31074452 0.0296862  0.5492497  0.02470057]
[0.  0.  0.8] by nn
[0.08561901 0.31074452 0.0296862  0.5492497  0.02470057]
[0.  0.  0.8] by nn
[0.08561901 0.31074452 0.0296862  0.5492497  0.02470057]
[0.  0.  0.8] by nn
[0.08561901 0.31074452 0.0296862  0.5492497  0.02470057]
[0.  0.  0.8] by nn
[0.08561901 0.31074452 0.0296862  0.5492497  0.02470057]
[0.  0.  0.8] by nn
[0.08561901 0.31074452 0.0296862  0.5492497  0.02470057]
[0.  0.  0.8] by nn
[0.08561901 0.31074452 0.0296862  0.5492497  0.02470057]
[0.  0.  0.8] by nn
[0.08561901 0.31074452 0.0296862  0.5492497  0.02470057]
[0.  0.  0.8] by nn
[0.08561901 0.31074452 0.0296862  0.5492497  0.02470057]
[0.  0.  0.8] by nn
[0.08561901 0.31074452 0.0296862  0.5492497  0.02470057]
[0.  0.  0.8] by nn

[0.08285601 0.28504258 0.03009192 0.5736594  0.02835007]
[0.  0.  0.8] by nn
[0.08285601 0.28504258 0.03009192 0.5736594  0.02835007]
[0.  0.  0.8] by nn
[0.08285601 0.28504258 0.03009192 0.5736594  0.02835007]
[0.  0.  0.8] by nn
[0.08285601 0.28504258 0.03009192 0.5736594  0.02835007]
[0.  0.  0.8] by nn
[0.08285601 0.28504258 0.03009192 0.5736594  0.02835007]
[0.  0.  0.8] by nn
[0.08285601 0.28504258 0.03009192 0.5736594  0.02835007]
[0.  0.  0.8] by nn
[0.08285601 0.28504258 0.03009192 0.5736594  0.02835007]
[0.  0.  0.8] by nn
[0.09001426 0.28628686 0.02949947 0.56548476 0.02871464]
[0.  0.  0.8] by nn
[0.09001426 0.28628686 0.02949947 0.56548476 0.02871464]
[0.  0.  0.8] by nn
[0.09001426 0.28628686 0.02949947 0.56548476 0.02871464]
[0.  0.  0.8] by nn
[0.09001426 0.28628686 0.02949947 0.56548476 0.02871464]
[0.  0.  0.8] by nn
[0.09001426 0.28628686 0.02949947 0.56548476 0.02871464]
[0.  0.  0.8] by nn
[0.09001426 0.28628686 0.02949947 0.56548476 0.02871464]
[0.  0.  0.8] by nn

[0.09551281 0.29988375 0.02688364 0.5485736  0.02914617]
[0.  0.  0.8] by nn
[0.07562865 0.28246102 0.0215372  0.58991504 0.03045811]
[0.  0.  0.8] by nn
[0.07562865 0.28246102 0.0215372  0.58991504 0.03045811]
[0.  0.  0.8] by nn
[0.07562865 0.28246102 0.0215372  0.58991504 0.03045811]
[0.  0.  0.8] by nn
[0.07562865 0.28246102 0.0215372  0.58991504 0.03045811]
[0.  0.  0.8] by nn
[0.07562865 0.28246102 0.0215372  0.58991504 0.03045811]
[0.  0.  0.8] by nn
[0.07562865 0.28246102 0.0215372  0.58991504 0.03045811]
[0.  0.  0.8] by nn
[0.07562865 0.28246102 0.0215372  0.58991504 0.03045811]
[0.  0.  0.8] by nn
[0.07562865 0.28246102 0.0215372  0.58991504 0.03045811]
[0.  0.  0.8] by nn
[0.07562865 0.28246102 0.0215372  0.58991504 0.03045811]
[0.  0.  0.8] by nn
[0.07562865 0.28246102 0.0215372  0.58991504 0.03045811]
[0.  0.  0.8] by nn
[0.08896209 0.29093382 0.02603848 0.56509334 0.0289723 ]
[0.  0.  0.8] by nn
[0.08896209 0.29093382 0.02603848 0.56509334 0.0289723 ]
[0.  0.  0.8] by nn